# Introduction to Modeling Dynamical System
Drake和Simulink一样使用动态系统描述
通过定义不同模块的输入，输出，操作等组成大的系统进行仿真



## 关于Drake底层的一些理解
1.动态连接

2.动态系统的仿真流程
https://www.mathworks.com/help/simulink/ug/simulating-dynamic-systems.html
3.关于协程


## System基类
关于System的函数推荐直接看[LeafSystem](https://drake.mit.edu/doxygen_cxx/classdrake_1_1systems_1_1_leaf_system.html)

### LeafSystem创建
默认初始化函数会创建一个没有input port, output port, parameters, events和scalar type的空模块

In [6]:
import numpy as np
from pydrake.all import LeafSystem
sys = LeafSystem()

### LeafSystem状态声明
LeafSystem通过指定变量个数声明状态
连续时间变量只有一组，但是离散时间变量每次调用Declare函数都会创建一组新的
当使用BasicVector创建时，会将输入作为默认值（0时刻的值）

In [7]:
from pydrake.all import BasicVector, AbstractValue
continuous_state_idx =sys.DeclareContinuousState(num_state_variables=1)
continuous_state_idx = sys.DeclareContinuousState(num_q=2,num_v=2,num_z=4)
continuous_state_idx = sys.DeclareContinuousState(model_vector=BasicVector(np.array([1,2,3])))
discrete_state_idx = sys.DeclareDiscreteState(model_vector=BasicVector(np.array([1,2,3,4,5])))
discrete_state_idx = sys.DeclareDiscreteState(model_vector=BasicVector(np.array([1,2,3])))
print(sys.num_continuous_states())
print(sys.num_discrete_state_groups())

3
2


### LeafSystem创建Context
通过Context获取仿真过程中系统的变化情况，包括时间，数值状态等。

In [15]:
sys_context = sys.CreateDefaultContext()
print(sys_context.get_continuous_state_vector())
print(sys_context.get_discrete_state(0))
sys.SetDefaultContext(sys_context)
print(sys_context.get_continuous_state_vector())
print(sys_context.get_discrete_state(0))


[1.0, 2.0, 3.0]
[1.0, 2.0, 3.0, 4.0, 5.0]
[1.0, 2.0, 3.0]
[1.0, 2.0, 3.0, 4.0, 5.0]
[1.0, 2.0, 3.0]
[1.0, 2.0, 3.0]


In [24]:
c_parent= sys.CreateDefaultContext()
c_child_normal = sys.GetMyContextFromRoot(c_parent)
c_child_normal2 = sys.GetMyContextFromRoot(c_parent)
c_child_mutable = sys.GetMyMutableContextFromRoot(c_parent)

c_child_normal.SetContinuousState(np.array([3,4,5]))
print(c_parent.get_continuous_state_vector())
print(c_child_normal2.get_continuous_state_vector())
print(c_child_mutable.get_continuous_state_vector())

[3.0, 4.0, 5.0]
[3.0, 4.0, 5.0]
[3.0, 4.0, 5.0]
